In [ ]:
from hugging_face.finetuning_of_the_pretrained_model.preprocessing import preprocess_function
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

tokenizer = AutoTokenizer.from_pretrained("deutsche-telekom/bert-multi-english-german-squad2")

# import torch
# from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
# # Same as before
checkpoint = "deutsche-telekom/bert-multi-english-german-squad2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
# sequences = [    "I've been waiting for a HuggingFace course my whole life.",    "This course is amazing!",]
# batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
# # This is new
# batch["labels"] = torch.tensor([1, 1])
# optimizer = AdamW(model.parameters())
# loss = model(**batch).loss
# loss.backward()
# optimizer.step()


data_collator = DefaultDataCollator()
dataset = load_dataset(
    path='./dataset'
    )

dataset["train"][0]

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01, )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, )
trainer.train()
